In [5]:
class Value:
    def __init__(self, data):
        self.data = data 

    def __repr__(self):
        return f"Value(data={self.data})"

Python __repr__() function returns the object representation in string format. This method is called when repr() function is invoked on the object. If possible, the string returned should be a valid Python expression that can be used to reconstruct the object again.
  
[Read More](https://www.digitalocean.com/community/tutorials/python-str-repr-functions)

In [8]:
# Create a Value object with value 3.0
a = Value(3.0)
a

Value(data=3.0)

In [9]:
a = Value(2.0)
b = Value(-3.0)
a,b

(Value(data=2.0), Value(data=-3.0))

### Addition

In [11]:
a + b

# Internally it calls a.__add__(b), but current Value object does not have __add__ function
# So it returns an error

TypeError: unsupported operand type(s) for +: 'Value' and 'Value'

In [12]:
class Value:
    def __init__(self, data):
        self.data = data 

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):   # a + b = a.__add__(b)
        out = Value(self.data + other.data)
        return out

In [13]:
a = Value(2.0)
b = Value(-3.0)

a + b

Value(data=-1.0)

### Multiplication

In [14]:
class Value:
    def __init__(self, data):
        self.data = data 

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        out = Value(self.data + other.data)
        return out 

    def __mul__(self, other):
        out = Value(self.data * other.data)
        return out

In [16]:
a = Value(2.0)
b = Value(-3.0)
c = Value(10.0)

# Original Equation 
d = a*b + c
d

Value(data=4.0)